In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
datetime_cols = [
    'reporting_dt',
    'fund_date', 'trade_opened_dt',
     'trade_close_dt', 
       'paymnt_condition_principal_terms_amt_dt',
       'paymnt_condition_interest_terms_amt_dt',       
       'paymnt_condition_grace_start_dt', 'paymnt_condition_grace_end_dt',
       'paymnt_condition_interest_payment_due_date',       
       'overall_val_credit_total_amt_date', 
       'month_aver_paymt_calc_date', 'collat_insured_insur_start_dt',
       'collat_insured_insur_end_dt', 'collat_insured_insur_fact_end_dt',       
       'collat_repay_dt', 'loan_indicator_dt', 'legal_items_court_act_dt', 
       'hold_dt', 'file_since_dt', 'last_updated_dt',
       'last_uploaded_dt', 'arrear_calc_date','due_arrear_start_dt',
       'due_arrear_calc_date','past_due_dt', 
       'past_due_calc_date', 'past_due_principal_missed_date',
       'past_due_int_missed_date']

In [4]:
# dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
df_bki = pd.read_csv('data/df_BKI_30k.csv', index_col=False, low_memory=False)

In [5]:
df_target = pd.read_csv('data/df_target_30k.csv', index_col=False, low_memory=False)

In [6]:
df_bki.head()

,application_id,client_id,equifax_id,reporting_dt,account_uid,fund_date,trade_owner_indic,trade_opened_dt,trade_trade_type_code,trade_loan_kind_code,trade_acct_type1,trade_is_consumer_loan,trade_has_card,trade_is_novation,trade_is_money_source,trade_close_dt,account_amt_credit_limit,account_amt_currency_code,account_amt_ensured_amt,coborrower_has_solidary,coborrower_solidary_num,paymnt_condition_principal_terms_amt,paymnt_condition_principal_terms_amt_dt,paymnt_condition_interest_terms_amt,paymnt_condition_interest_terms_amt_dt,paymnt_condition_terms_frequency,paymnt_condition_min_paymt,paymnt_condition_grace_start_dt,paymnt_condition_grace_end_dt,paymnt_condition_interest_payment_due_date,overall_val_credit_total_amt,overall_val_credit_total_monetary_amt,overall_val_credit_total_amt_date,month_aver_paymt_aver_paymt_amt,month_aver_paymt_calc_date,has_collaterals,has_guarantees,has_indie_guarantees,collat_insured_insur_sign,collat_insured_insur_limit,collat_insured_currency_code,collat_insured_has_franchise,collat_insured_insur_start_dt,collat_insured_insur_end_dt,collat_insured_insur_fact_end_dt,collat_insured_insur_end_reason,colat_repaid,collat_repay_code,collat_repay_dt,collat_repay_amt,loan_indicator,loan_indicator_dt,legal_items_has_legal_dispute,legal_items_court_act_dt,legal_items_court_act_effect_code,hold_code,hold_dt,file_since_dt,last_updated_dt,last_uploaded_dt,arrear_sign,arrear_start_amt_outstanding,arrear_last_payment_due_code,arrear_amt_outstanding,arrear_principal_outstanding,arrear_int_outstanding,arrear_other_amt_outstanding,arrear_calc_date,arrear_unconfirm_grace,due_arrear_start_dt,due_arrear_last_payment_due_code,due_arrear_amt_outstanding,due_arrear_principal_outstanding,due_arrear_int_outstanding,due_arrear_other_amtoutstanding,due_arrear_calc_date,past_due_dt,past_due_last_payment_due_code,past_due_amt_past_due,past_due_principal_amt_past_due,past_due_int_amt_past_due,past_due_other_amt_past_due,past_due_calc_date,past_due_principal_missed_date,past_due_int_missed_date,delay5,delay30,delay60,delay90,delay_more,cred_max_overdue,attr_value
0,51859026,15984285,9344571,2023-08-01,902086582865039990,2014-07-11,1.0,2014-07-11,1.0,1.0,15.0,1.0,0.0,0.0,0.0,2017-07-11,240000.0,RUB,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.060,NaN,2017-09-20,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,2017-09-20,NaN,NaN,NaN,NaN,NaN,2014-07-11,2017-09-20,2017-10-01,0.0,240000.00,1.0,0.00,0.00,0.00,0.00,2017-09-20,NaN,NaN,1.0,0.00,0.00,0.00,0.00,2017-09-20,NaN,1.0,0.0,0.0,0.0,0.0,2017-09-20,NaN,NaN,2,2,0,1,16,113333.27,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
1,51859359,15984353,9344781,2023-08-01,654289381013228862,2015-02-13,1.0,2015-02-13,1.0,1.0,15.0,1.0,0.0,0.0,0.0,2017-02-13,126400.0,RUB,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.810,NaN,2020-10-26,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,2019-09-01,1.0,3.0,2020-10-26,2015-02-13,2020-10-26,2020-10-31,0.0,126400.00,1.0,0.00,0.00,0.00,0.00,2020-10-26,NaN,NaN,1.0,0.00,0.00,0.00,0.00,2020-10-26,NaN,1.0,0.0,0.0,0.0,0.0,2020-10-26,NaN,NaN,0,1,0,0,13,77541.87,SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSAAAAAAAAAAAA...
2,51859359,15984353,9344781,2023-08-01,741680445545450197,2012-03-21,1.0,2012-03-21,1.0,5.0,19.0,1.0,0.0,0.0,1.0,2100-01-01,90000.0,RUB,NaN,NaN,NaN,845.48,2023-07-31,0.00,NaN,3.0,NaN,NaN,NaN,NaN,0.100,101457.3,2021-06-15,845.0,2023-06-30,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-15,2023-06-30,2023-07-07,1.0,90000.00,0.0,101457.30,101457.30,0.00,0.00,2023-06-30,0.0,NaN,0.0,0.00,NaN,NaN,NaN,2023-06-30,2017-02-20,0.0,101457.3,101457.3,0.0,0.0,2023-06-30,2017-02-19,NaN,0,0,0,0,1,101457.30,--543100000AAAAAAAA98765432-------------------...
3,51859863,15984478,9345100,2023-08-01,634072954671362970,2023-05-10,1.0,2023-05-10,1.0,7.0,99.0,1.0,1.0,0.0,1.0,2033-05-10,60000.0,RUB,NaN,0.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,2023-07-

In [7]:
df_bki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481335 entries, 0 to 481334
Data columns (total 92 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   application_id                              481335 non-null  int64  
 1   client_id                                   481335 non-null  int64  
 2   equifax_id                                  481335 non-null  int64  
 3   reporting_dt                                481335 non-null  object 
 4   account_uid                                 481335 non-null  int64  
 5   fund_date                                   479389 non-null  object 
 6   trade_owner_indic                           479668 non-null  float64
 7   trade_opened_dt                             481334 non-null  object 
 8   trade_trade_type_code                       481334 non-null  float64
 9   trade_loan_kind_code                        481190 non-null  float64
 

In [8]:
columns_wo_ids_and_dates = ['trade_owner_indic', 'trade_trade_type_code', 'trade_loan_kind_code', 'trade_acct_type1','trade_is_consumer_loan', 'trade_has_card', 'trade_is_novation','trade_is_money_source', 'account_amt_currency_code', 'coborrower_has_solidary', 'coborrower_solidary_num', 'paymnt_condition_terms_frequency', 'has_collaterals', 'has_guarantees','has_indie_guarantees', 'collat_insured_insur_sign', 'collat_insured_currency_code','collat_insured_has_franchise','collat_insured_insur_end_reason', 'colat_repaid', 'collat_repay_code', 'loan_indicator','legal_items_has_legal_dispute','legal_items_court_act_effect_code','arrear_sign', 'arrear_last_payment_due_code',  'arrear_unconfirm_grace', 'due_arrear_last_payment_due_code', 'past_due_last_payment_due_code']

In [9]:
for col in df_bki[columns_wo_ids_and_dates]:
    print('-------------------------')
    print(f'{col}:')
    print(df_bki[col].value_counts(dropna=False))

-------------------------
trade_owner_indic:
trade_owner_indic
1.0     479414
NaN       1667
2.0        135
99.0       112
4.0          7
Name: count, dtype: int64
-------------------------
trade_trade_type_code:
trade_trade_type_code
1.0    481185
4.0       135
5.0         7
2.0         7
NaN         1
Name: count, dtype: int64
-------------------------
trade_loan_kind_code:
trade_loan_kind_code
3.0     306593
1.0     100276
5.0      50829
7.0      10207
4.0      10127
99.0      1611
2.0       1545
NaN        145
6.0          2
Name: count, dtype: int64
-------------------------
trade_acct_type1:
trade_acct_type1
19.0    275732
99.0    140574
15.0     46589
14.0     10349
1.0       2252
8.0       1778
17.0       912
2.5        749
9.0        688
NaN        528
2.4        454
18.0       278
16.1        98
16.3        94
4.1         65
4.7         50
2.7         40
7.0         31
4.5         22
2.1         17
12.0        15
2.2         13
3.0          5
13.0         1
4.2          1
Nam

In [10]:
df_target['target'].value_counts(dropna=False)

target
0    23605
1     6395
Name: count, dtype: int64

In [11]:
df_bki['client_id'].value_counts()

client_id
7116824     492
6683653     383
2437607     346
3481438     313
9983244     309
14083823    297
1799374     295
2620656     288
7432435     285
7881236     284
10313790    272
1920402     272
8550401     257
16180446    257
7577895     256
16136482    255
3254333     254
14586619    250
14694461    247
14367665    233
16024491    230
10487917    224
9883344     221
16001612    218
12561530    210
12646911    207
15506808    206
3818990     201
9322142     201
16234771    200
7285198     200
13479666    200
15028104    197
9536105     196
15971119    192
16033199    192
15440724    192
15943170    191
14665118    190
15826260    188
6646229     187
14523669    186
3293412     185
3662029     182
831067      182
2601065     181
16222886    180
16247977    179
15992841    179
2128730     178
5126676     177
10320899    176
15386021    175
9668319     175
8306024     170
15490943    169
5220609     169
9158175     168
15968683    168
8029119     167
11864875    167
11397140    16

In [26]:
df_bki['application_id'].value_counts()

application_id
53123153    492
52175920    383
53031443    346
52423025    313
51787828    309
52176301    297
52362901    295
52797693    288
53025488    285
52443817    284
51966316    272
52678272    272
52226065    257
52788232    257
52822435    256
52583201    255
52357068    254
53040868    250
51650163    247
52758109    233
52057300    230
52011097    224
52769310    221
51946569    218
52307039    210
52325801    207
52464478    206
52365634    201
51788520    201
53040372    200
52735321    200
51634312    200
52759364    197
52900970    196
51801010    192
52100583    192
52909873    192
51670421    191
52326117    190
52597429    188
52287369    187
52279176    186
51844196    185
53129703    182
53024141    182
51651453    181
52983417    180
53108330    179
52762951    179
52945653    178
52337134    177
52934230    176
52627957    175
52737895    175
52770708    170
51591305    169
52536624    169
52628693    168
51789447    168
52503185    167
52359523    167
52052753 

In [13]:
df_test = pd.get_dummies(df_bki.loc[(df_bki['application_id'] == 53123153) | (df_bki['application_id'] == 52175920)][['application_id', 'trade_acct_type1', 'trade_loan_kind_code']], columns=['trade_acct_type1', 'trade_loan_kind_code'], dtype=int)

In [ ]:
df_test

In [17]:
df_test.groupby(['application_id']).sum()

,trade_acct_type1_1.0,trade_acct_type1_4.7,trade_acct_type1_8.0,trade_acct_type1_9.0,trade_acct_type1_14.0,trade_acct_type1_15.0,trade_acct_type1_19.0,trade_acct_type1_99.0,trade_loan_kind_code_1.0,trade_loan_kind_code_2.0,trade_loan_kind_code_3.0,trade_loan_kind_code_4.0,trade_loan_kind_code_5.0,trade_loan_kind_code_7.0,trade_loan_kind_code_99.0
application_id,,,,,,,,,,,,,,,
52175920,2,1,6,1,1,2,311,59,3,1,366,2,7,3,1
53123153,1,0,0,0,1,2,476,12,2,0,485,0,5,0,0


In [ ]:
df_bki['trade_opened_dt'].value_counts().sort_index(ascending=True)

trade_opened_dt
2002-12-18       1
2003-05-27       1
2003-07-18       1
2003-09-02       1
2004-04-19       1
2004-05-06       1
2004-05-27       1
2004-05-28       1
2004-06-09       1
2004-07-29       1
2004-08-05       1
2004-10-14       1
2004-11-18       1
2004-11-30       1
2004-12-12       1
2004-12-13       1
2005-01-08       1
2005-01-20       1
2005-01-30       1
2005-02-11       1
2005-02-12       1
2005-02-24       1
2005-02-25       1
2005-03-01       1
2005-03-03       1
2005-03-07       1
2005-03-19       1
2005-03-25       1
2005-04-01       1
2005-04-22       1
2005-04-25       1
2005-05-15       2
2005-05-26       1
2005-05-27       1
2005-05-28       1
2005-06-05       1
2005-06-10       1
2005-06-11       2
2005-06-13       1
2005-06-14       1
2005-06-15       1
2005-06-25       1
2005-06-28       1
2005-07-06       1
2005-07-12       1
2005-07-21       1
2005-07-26       1
2005-08-01       1
2005-08-03       1
2005-08-11       2
2005-08-16       1
2005-08-23     

In [28]:
df_bki['reporting_dt'].value_counts().sort_index(ascending=True)

reporting_dt
2023-07-19      959
2023-07-20     5360
2023-07-21     5230
2023-07-22     5309
2023-07-23     5747
2023-07-24     6510
2023-07-25     5019
2023-07-26     4607
2023-07-28     5420
2023-07-29     6254
2023-07-30     6039
2023-07-31     7319
2023-08-01     6102
2023-08-02     5139
2023-08-03     5860
2023-08-04     5151
2023-08-05     4793
2023-08-06     6231
2023-08-07     6924
2023-08-08     5785
2023-08-09     5996
2023-08-10     5671
2023-08-11     5203
2023-08-12     6768
2023-08-13     7343
2023-08-14     5993
2023-08-15     6748
2023-08-16     6627
2023-08-17     8356
2023-08-18    10381
2023-08-19    10043
2023-08-20    11882
2023-08-21     8321
2023-08-22     8267
2023-08-23    13974
2023-08-24    10790
2023-08-25    10870
2023-08-26    10682
2023-08-27    12030
2023-08-28     9679
2023-08-29    11101
2023-08-30    12693
2023-08-31    11260
2023-09-01    10007
2023-09-02     9573
2023-09-03     7918
2023-09-04    10764
2023-09-05    10460
2023-09-06     9479
2023-09

In [37]:
df_bki[(df_bki['application_id'] == 53123153)]['trade_opened_dt'].value_counts().sort_index(ascending=True)

trade_opened_dt
2013-04-11    1
2013-12-25    1
2014-05-06    1
2014-07-04    1
2018-05-15    1
2018-05-16    1
2018-05-23    1
2018-05-25    1
2018-05-28    1
2018-05-29    1
2018-05-30    1
2018-06-04    1
2018-06-05    1
2018-06-12    1
2018-06-15    1
2018-06-16    2
2018-06-19    2
2018-06-20    2
2018-06-26    2
2018-07-10    1
2018-07-14    2
2018-07-15    2
2018-07-19    1
2018-07-26    1
2018-08-01    1
2018-08-15    2
2018-08-16    1
2018-08-21    1
2018-08-22    1
2018-08-24    1
2018-08-25    1
2018-08-26    2
2018-08-27    1
2018-08-28    1
2018-08-29    1
2018-09-08    1
2018-09-10    1
2018-09-11    1
2018-09-12    1
2018-09-17    1
2018-09-19    1
2018-09-20    1
2018-09-21    1
2018-09-22    2
2018-09-23    1
2018-09-25    3
2018-09-26    2
2018-09-29    2
2018-09-30    1
2018-10-01    1
2018-10-02    2
2018-10-04    2
2018-10-06    1
2018-10-08    1
2018-10-09    2
2018-10-10    2
2018-10-15    1
2018-10-16    2
2018-10-17    1
2018-10-20    2
2018-10-21    1
2018-10-

In [38]:
df_bki[(df_bki['application_id'] == 53123153)]['trade_close_dt'].value_counts().sort_index(ascending=True)

trade_close_dt
2018-05-06    1
2018-05-25    1
2018-05-29    1
2018-05-31    1
2018-06-03    1
2018-06-04    2
2018-06-05    1
2018-06-11    1
2018-06-18    1
2018-06-25    1
2018-06-26    3
2018-07-04    1
2018-07-10    1
2018-07-16    2
2018-07-19    1
2018-07-20    2
2018-07-21    1
2018-07-25    1
2018-07-26    1
2018-08-06    1
2018-08-08    1
2018-08-10    1
2018-08-13    1
2018-08-14    1
2018-08-24    1
2018-08-26    1
2018-08-28    1
2018-09-10    1
2018-09-11    3
2018-09-14    1
2018-09-18    2
2018-09-20    1
2018-09-24    1
2018-09-25    2
2018-09-27    1
2018-09-29    2
2018-10-01    1
2018-10-02    1
2018-10-10    1
2018-10-11    1
2018-10-12    1
2018-10-13    1
2018-10-15    1
2018-10-16    2
2018-10-17    1
2018-10-22    4
2018-10-23    3
2018-10-24    1
2018-10-25    3
2018-10-27    1
2018-10-28    1
2018-10-29    2
2018-10-30    5
2018-11-08    1
2018-11-10    3
2018-11-12    2
2018-11-13    2
2018-11-14    3
2018-11-17    2
2018-11-24    1
2018-11-26    1
2018-11-2

In [ ]:
df_bki[(df_bki['application_id'] == 53123153)][['trade_opened_dt', 'trade_close_dt']].sort_values('trade_opened_dt').head()

In [12]:
df_test1 = pd.pivot_table(df_bki[['application_id', 'account_amt_credit_limit', 'trade_loan_kind_code']], values=['account_amt_credit_limit'], index=['application_id'], columns=['trade_loan_kind_code'], aggfunc='sum')

In [15]:
df_test1 = df_test1.fillna(0.0)

In [18]:
df_test1.head()

account_amt_credit_limit                           \
trade_loan_kind_code                     1.0  2.0        3.0      4.0    
application_id                                                           
51534101                            387063.66  0.0  110394.95      0.0   
51534210                              5300.00  0.0   88329.00      0.0   
51534377                                 0.00  0.0   17000.00      0.0   
51534467                            119726.58  0.0       0.00  15000.0   
51534573                            497962.00  0.0  211845.38  16856.0   

                                                    
trade_loan_kind_code      5.0  6.0       7.0  99.0  
application_id                                      
51534101                   0.0  0.0       0.0  0.0  
51534210                   0.0  0.0       0.0  0.0  
51534377               56000.0  0.0       0.0  0.0  
51534467               15000.0  0.0       0.0  0.0  
51534573              188000.0  0.0  190000.0  0.0

In [20]:
df_test1.loc[53123153]

                          trade_loan_kind_code
account_amt_credit_limit  1.0                      799000.00
                          2.0                           0.00
                          3.0                     7899652.09
                          4.0                           0.00
                          5.0                      270000.10
                          6.0                           0.00
                          7.0                           0.00
                          99.0                          0.00
Name: 53123153, dtype: float64

In [21]:
df_test2 = pd.pivot_table(df_bki[['application_id', 'account_amt_credit_limit', 'trade_loan_kind_code']], values=['account_amt_credit_limit'], index=['application_id'], columns=['trade_loan_kind_code'], aggfunc='mean')

In [23]:
df_test2 = df_test2.fillna(0.0)

In [24]:
df_test2.head()

account_amt_credit_limit                              \
trade_loan_kind_code                     1.0  2.0           3.0      4.0    
application_id                                                              
51534101                        129021.220000  0.0   7885.353571      0.0   
51534210                          5300.000000  0.0   5520.562500      0.0   
51534377                             0.000000  0.0   5666.666667      0.0   
51534467                         59863.290000  0.0      0.000000  15000.0   
51534573                         45269.272727  0.0  30263.625714  16856.0   

                                                       
trade_loan_kind_code     5.0  6.0           7.0  99.0  
application_id                                         
51534101                  0.0  0.0      0.000000  0.0  
51534210                  0.0  0.0      0.000000  0.0  
51534377              28000.0  0.0      0.000000  0.0  
51534467              15000.0  0.0      0.000000  0.0  
51534573              37600.0  0.0  63333.333333  0.0